In [43]:
import os
import numpy as np
from sklearn.decomposition import PCA
import random
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path
from scipy.signal import resample


In [44]:
dataset = 'nifeadb'

In [45]:
import os
import numpy as np
from sklearn.decomposition import PCA

def univariate(dataset_name):
    data_dir = os.path.join(processed_data_path, dataset_name + "_preprocessed.npy")
    data = np.load(data_dir, allow_pickle=True)

    # Define the path to save the new dataset
    

    # Initialize a list for storing modified entries
    univariate_data = []
    retained_variances = []  # List to store retained variance for each entry

    for entry in data:
        # Copy the entry to preserve metadata
        new_entry = entry.copy()

        # Extract the signal matrix (shape: (599999, 6))
        signal_matrix = entry['signal']
        
        # Apply PCA to reduce from 6D to 1D
        pca = PCA(n_components=1)
        reduced_signal = pca.fit_transform(signal_matrix)  # Shape: (599999, 1)
        
        # Calculate retained variance
        retained_variance = np.sum(pca.explained_variance_ratio_)
        retained_variances.append(retained_variance)

        print(f"Retained Variance for {entry['record_name']}: {retained_variance:.6f}")
        
        # Flatten to (599999,) to retain time-series format
        new_entry['signal'] = reduced_signal.flatten()

        # Append modified entry to the new dataset
        univariate_data.append(new_entry)

    # Compute the average retained variance for the dataset
    avg_retained_variance = np.mean(retained_variances)
    print(f"\nAverage Retained Variance for {dataset_name}: {avg_retained_variance:.6f}")

    # Convert to a NumPy array and save
    
    return univariate_data

univariate_data = univariate(dataset)


Retained Variance for ARR_01: 0.838394
Retained Variance for ARR_02: 0.531885
Retained Variance for ARR_03: 0.463051
Retained Variance for ARR_04: 0.531407
Retained Variance for ARR_05: 0.802113
Retained Variance for ARR_06: 0.680092
Retained Variance for ARR_07: 0.801713
Retained Variance for ARR_08: 0.842037
Retained Variance for ARR_09: 0.832517
Retained Variance for ARR_10: 0.626628
Retained Variance for ARR_11: 0.826809
Retained Variance for ARR_12: 0.864628
Retained Variance for NR_01: 0.821890
Retained Variance for NR_02: 0.679275
Retained Variance for NR_03: 0.509599
Retained Variance for NR_04: 0.617685
Retained Variance for NR_05: 0.670962
Retained Variance for NR_06: 0.667240
Retained Variance for NR_07: 0.818575
Retained Variance for NR_08: 0.679510
Retained Variance for NR_09: 0.637419
Retained Variance for NR_10: 0.480174
Retained Variance for NR_11: 0.411022
Retained Variance for NR_12: 0.415964
Retained Variance for NR_13: 0.531885
Retained Variance for NR_14: 0.818575


In [46]:
def downsample_target(univariate_data, target_fs=20):
    """
    Downsamples all signals OF TARGET SET in univariate_data to the target frequency (default: 20Hz).
    
    Parameters:
    - univariate_data (list of dicts): Each entry should have 'metadata' with 'fs' (sampling rate) and 'signal'.
    - target_fs (int): The target sampling frequency (default is 20 Hz).
    
    Returns:
    - downsampled_data (list of dicts): Same structure as input but with downsampled signals.
    """
    downsampled_data = []
    
    for entry in univariate_data:
        original_fs = entry['fs']
        print('óriginaloriginal_fs', original_fs)
        signal = entry['signal']
        
        # Compute the new length after downsampling
        new_length = int(len(signal) * target_fs / original_fs)
        print(target_fs, original_fs)
        print(len(signal), new_length)
        
        # Resample the signal
        downsampled_signal = resample(signal, new_length)
        print('old signal lengt', len(signal))
        print('new length', len(downsampled_signal))
        # Store the downsampled signal with updated metadata
        downsampled_entry = {
            'fs': target_fs,
            'signal': downsampled_signal,
            'record_name': entry['record_name']
        }
        downsampled_data.append(downsampled_entry)
    save_dir = os.path.join(univariate_data_path, dataset + "_univariate.npy")
    np.save(save_dir, np.array(univariate_data, dtype=object))

    print(f"PCA transformation complete. Saved as '{save_dir}'.")
    
    return downsampled_data

def downsample_signal(univariate_data, target_fs=20):
    """
    Downsamples all signals in univariate_data to the target frequency (default: 20Hz).
    
    Parameters:
    - univariate_data (list of dicts): Each entry should have 'metadata' with 'fs' (sampling rate) and 'signal'.
    - target_fs (int): The target sampling frequency (default is 20 Hz).
    
    Returns:
    - downsampled_data (list of dicts): Same structure as input but with downsampled signals.
    """
    downsampled_data = []
    
    for entry in univariate_data:
        original_fs = entry['metadata']['fs']
        print('óriginaloriginal_fs', original_fs)
        signal = entry['signal']
        
        # Compute the new length after downsampling
        new_length = int(len(signal) * target_fs / original_fs)
        print(target_fs, original_fs)
        print(len(signal), new_length)
        
        # Resample the signal
        downsampled_signal = resample(signal, new_length)
        print('old signal lengt', len(signal))
        print('new length', len(downsampled_signal))
        # Store the downsampled signal with updated metadata
        downsampled_entry = {
            'metadata': {**entry['metadata'], 'fs': target_fs},
            'signal': downsampled_signal,
            'record_name': entry['record_name']
        }
        downsampled_data.append(downsampled_entry)
    save_dir = os.path.join(univariate_data_path, dataset + "_univariate.npy")
    np.save(save_dir, np.array(univariate_data, dtype=object))

    print(f"PCA transformation complete. Saved as '{save_dir}'.")
    
    return downsampled_data

if dataset == 'target':
    downsampled_data = downsample_target(univariate_data)
else: 
    downsampled_data = downsample_signal(univariate_data)
print(len(univariate_data[1]['signal']))
print(len(downsampled_data[1]['signal']))

# print(data[1])
print(downsampled_data[1])

óriginaloriginal_fs 1000
20 1000
480052 9601


old signal lengt 480052
new length 9601
óriginaloriginal_fs 1000
20 1000
484024 9680
old signal lengt 484024
new length 9680
óriginaloriginal_fs 1000
20 1000
637897 12757
old signal lengt 637897
new length 12757
óriginaloriginal_fs 1000
20 1000
485207 9704
old signal lengt 485207
new length 9704
óriginaloriginal_fs 1000
20 1000
361689 7233
old signal lengt 361689
new length 7233
óriginaloriginal_fs 500
20 500
901521 36060
old signal lengt 901521
new length 36060
óriginaloriginal_fs 500
20 500
249423 9976
old signal lengt 249423
new length 9976
óriginaloriginal_fs 500
20 500
600803 24032
old signal lengt 600803
new length 24032
óriginaloriginal_fs 500
20 500
272900 10916
old signal lengt 272900
new length 10916
óriginaloriginal_fs 1000
20 1000
487118 9742
old signal lengt 487118
new length 9742
óriginaloriginal_fs 1000
20 1000
485178 9703
old signal lengt 485178
new length 9703
óriginaloriginal_fs 1000
20 1000
485152 9703
old signal lengt 485152
new length 9703
óriginaloriginal_fs 1000


In [47]:
# import numpy as np
# import matplotlib.pyplot as plt

# data_dir = os.path.join(processed_data_path, dataset + "_preprocessed.npy")
# or_dat = np.load(data_dir, allow_pickle=True)
# # Select an instance
# instance = or_dat[1]  # Using the provided index 1
# print('original fs', instance['metadata']['fs'])
# record_name = instance['record_name']
# original_signals = instance['signal']  # Multivariate signals
# print(original_signals)
# fs = instance['metadata']['fs']

# # Find the corresponding univariate signal
# univariate_instance = univariate_data[1]
# print(instance['record_name'], univariate_instance['record_name'])
# univariate_signal = univariate_instance['signal']

# # Determine the number of channels
# num_channels = original_signals.shape[1]
# print(num_channels)
# sequence_length = original_signals.shape[0]

# # Create subplots (num_channels + 1 to include the univariate signal)
# fig, axs = plt.subplots(num_channels + 1, 1, figsize=(12, 2 * (num_channels + 1)), sharex=True)
# time_axis = np.arange(sequence_length) / fs  # Convert to seconds

# # Plot all original channels
# title = f"Original {num_channels}-Channel Signals & PCA-Reduced Univariate Signal ({record_name})"
# for i in range(num_channels):
#     axs[i].plot(time_axis, original_signals[:, i], label=f'Channel {i+1}', color='b', alpha=0.7)
#     axs[i].legend()
#     axs[i].set_ylabel("Amplitude")

# # Plot the univariate signal in a distinct color
# axs[num_channels].plot(time_axis, univariate_signal, label="Univariate Signal (PCA)", color='r')
# axs[num_channels].legend()
# axs[num_channels].set_ylabel("Amplitude")
# axs[num_channels].set_xlabel("Time (seconds)")

# fig.suptitle(title)
# plt.tight_layout()
# plt.show()


In [48]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Select the downsampled signal
# downsampled_signal = downsampled_data[1]['signal']
# record_name = downsampled_data[1]['record_name']

# # Create time axis assuming uniform sampling
# downsampled_fs = 20  # The new sampling frequency
# downsampled_time_axis = np.arange(len(downsampled_signal)) / downsampled_fs

# # Plot the downsampled signal
# plt.figure(figsize=(12, 4))
# plt.plot(downsampled_time_axis, downsampled_signal, label="Downsampled Signal", color='g')
# plt.xlabel("Time (seconds)")
# plt.ylabel("Amplitude")
# plt.title(f"Downsampled Signal ({record_name}) at 20Hz")
# plt.legend()
# plt.show()


In [49]:
# import numpy as np
# import matplotlib.pyplot as plt
# import os
# import random

# # Load dataset
# data_path = os.path.join(univariate_data_path, dataset + "_univariate.npy")
# data = np.load(data_path, allow_pickle=True)

# # Select a random instance for visualization
# random_instance = random.choice(data)
# univariate_signal = random_instance['signal']  # Univariate after PCA

# # Load the original dataset before PCA
# original_data_path = os.path.join(processed_data_path, dataset + "_preprocessed.npy")
# original_data = np.load(original_data_path, allow_pickle=True)

# # Find the corresponding original signal
# record_name = random_instance['record_name']
# original_instance = next(entry for entry in original_data if entry['record_name'] == record_name)
# original_multivariate_signal = original_instance['signal']  # Shape: (sequence_length, num_channels)

# num_channels = original_multivariate_signal.shape[1]  

# # Create subplots (num_channels + 1 to include the univariate signal)
# fig, axs = plt.subplots(num_channels + 1, 1, figsize=(12, 2 * (num_channels + 1)), sharex=True)

# time_axis = np.arange(original_multivariate_signal.shape[0])

# # Plot all channels dynamically
# for i in range(num_channels):
#     axs[i].plot(time_axis, original_multivariate_signal[:, i], label=f'Channel {i+1}', color='b', alpha=0.7)
#     axs[i].legend()
#     axs[i].set_ylabel("Amplitude")

# # Plot the univariate signal in a distinct color
# axs[num_channels].plot(time_axis, univariate_signal, label="Univariate Signal (PCA)", color='r')
# axs[num_channels].legend()
# axs[num_channels].set_ylabel("Amplitude")
# axs[num_channels].set_xlabel("Time")

# fig.suptitle(f"Original {num_channels}-Channel Signal & PCA-Reduced Univariate Signal")
# plt.tight_layout()
# plt.show()
